In [67]:
import os
import pandas as pd

In [68]:
from modules.config import DATADIR
from modules.utils import load_json, print_groupby

In [69]:
# Load factory contract addresses for each DAO framework
df_factories = pd.read_csv(os.path.join(DATADIR, 'framework_factory_contract_addresses.csv'), index_col=False)
df_factories.set_index('version', inplace=True)
df_factories['abis'] = None

# Load abis for each DAO framework
for version in df_factories.index:
    abis_full = load_json(os.path.join(DATADIR, f"trueblocks_framework_abis_{version}.json"))['data']
    abis = []
    for abi in abis_full:
        abiDict = {k: abi[k] for k in ['name', 'type']}
        abiDict['inputs'] = {v['name']: v['type'] for v in abi.get('inputs', {})}
        abis.append(abiDict)
    df_factories.at[version, 'abis'] = abis

print(df_factories.index[0])
df_factories['abis'][0]

aragon_0.8.1


[{'name': 'baseACL', 'type': 'function', 'inputs': {}},
 {'name': 'DeployEVMScriptRegistry',
  'type': 'event',
  'inputs': {'reg': 'address'}},
 {'name': 'newDAO', 'type': 'function', 'inputs': {'_root': 'address'}},
 {'name': 'DeployDAO', 'type': 'event', 'inputs': {'dao': 'address'}},
 {'name': 'regFactory', 'type': 'function', 'inputs': {}},
 {'name': 'baseKernel', 'type': 'function', 'inputs': {}}]

In [70]:
# Load transaction data from all factory contracts by framework version
df_transactions = pd.DataFrame()
files = [f for f in os.listdir(DATADIR) if f.startswith('trueblocks_export')]
for f in files:
    version = "_".join(os.path.splitext(f)[0].split('_')[2:])
    dataDir = load_json(os.path.join(DATADIR, f))
    dataList = [{'id': f"{version}_{k}",
                 'framework': df_factories.at[version, 'framework'],
                 'version': version,
                 'functionName': v.get('articulatedTx', {}).get('name'),
                 'functionInputs': v.get('articulatedTx', {}).get('inputs'),
                 'transactionHash': v.get('hash')}
                for k, v in dataDir.items() if int(k) != 0]
    df_tmp = pd.DataFrame(dataList)
    df_transactions = df_transactions.append(df_tmp, ignore_index=True)

gb = df_transactions.groupby('version')
print(gb['id'].count())

version
aragon_0.6            257
aragon_0.7            241
aragon_0.8            149
aragon_0.8.1          149
aragon_court          310
daohaus_molochv2.1    241
daostack_alchemy      125
openlaw_tribute       107
Name: id, dtype: int64


In [71]:
# Load contract creation trace data from all factory contracts by framework version
files = [f for f in os.listdir(DATADIR) if f.startswith('trueblocks_trace')]
df_traces = pd.DataFrame()
for f in files:
    version = "_".join(os.path.splitext(f)[0].split('_')[2:])
    dataDir = load_json(os.path.join(DATADIR, f))
    # Get contracts created
    dataList = []
    for key, fullTrace in dataDir.items():
        if int(key) != 0:
            dataList.append({'id': f"{version}_{key}",
                            'version': version,
                            **fullTrace})
    df_tmp = pd.DataFrame(dataList)
    df_traces = df_traces.append(df_tmp, ignore_index=True)

gb = df_traces.groupby('version')
print(gb['id'].count())

version
aragon_0.8            149
aragon_0.8.1          149
daohaus_molochv2.1    256
daostack_alchemy      122
openlaw_tribute       107
Name: id, dtype: int64


In [76]:
# Merge transaction data with contract creation data
df_data = pd.merge(df_transactions, df_traces, on ='id', how ="outer", suffixes=(None, "_y"))
df_data.drop(columns=['version_y'], inplace=True)
print(df_data.columns)
df_data.to_csv(os.path.join(DATADIR, "trueblocks_factory_transactions.csv"))

gb = df_data.groupby('version')
print(gb['id'].count())

Index(['id', 'framework', 'version', 'functionName', 'functionInputs',
       'transactionHash', 'contractsCreated', 'nContracts'],
      dtype='object')
version
aragon_0.6            257
aragon_0.7            241
aragon_0.8            149
aragon_0.8.1          149
aragon_court          310
daohaus_molochv2.1    241
daostack_alchemy      125
openlaw_tribute       107
Name: id, dtype: int64


In [74]:
# So what's in the transaction history for each framework?
for key, item in gb:
    print(key)
    df_group = pd.DataFrame(gb.get_group(key))
    df_create = df_group[df_group["contractsCreated"] == []]
    print(f"{len(df_create.index)} transactions didn't create address")
    print(df_create)

aragon_0.6


ValueError: ('Lengths must match to compare', (257,), (0,))